In [25]:
import numpy as np
import numpy.linalg as la
import numpy.random as rnd
import matplotlib.pyplot as plt
import scipy
import networkx as nx

In [26]:
# hay que setear tambien esta seed porque networkx la utiliza
# https://networkx.org/documentation/stable/reference/randomness.html
import random
random.seed(2023)

# Generamos una matriz A
rnd.seed(2023)

nt = [100, 100]
p = [[0.5, 0.2],[0.2, 0.5]]
n = np.sum(nt)
d=2

g = nx.stochastic_block_model(nt,p)
A = nx.to_numpy_array(g)

def resolver_con_svd(A, d):
    UA, SA, VAt = scipy.linalg.svd(A)
    XA = UA[:,0:d].dot(np.diag(np.sqrt(SA[0:d])))
    return XA

def f(X):
  return la.norm((A - X@X.T),ord='fro')**2

In [27]:
X_svd = resolver_con_svd(A,d)
print('f(X_svd) = ',f(X_svd))

f(X_svd) =  8045.311152055572


In [28]:
# Generar una matriz aleatoria
C = rnd.rand(d, d)
# Obtener la descomposición QR de la matriz
Q, R = np.linalg.qr(C)

XC = X_svd @ Q

# Sustituyendo XC en la función f(X) tenemos que
print('f(XC) = ',f(XC))
print('f(X_svd) = ',f(X_svd))

# Verificamos que sean 2 soluciones distintas
assert not np.array_equal(XC,X_svd)

f(XC) =  8045.311152055572
f(X_svd) =  8045.311152055572


In [6]:
def chequear_convexidad(X1, X2, alpha=0.5):
    # Valor de la función objetivo para X1, X2 y su combinación convexa
    f_X1 = f(X1)
    f_X2 = f(X2)
    f_comb = f(alpha * X1 + (1 - alpha) * X2)
    print(f"{f_comb} <= {alpha * f_X1 + (1 - alpha) * f_X2}")

    # Comprobación de convexidad
    if f_comb > alpha * f_X1 + (1 - alpha) * f_X2:
        print("La función objetivo no es convexa.")
    else:
        print("La función objetivo es convexa.")

chequear_convexidad(X_svd, XC)

11612.797719750053 <= 8045.311152055572
La función objetivo no es convexa.


In [29]:
def grad_f(X):
    return 4*(-A+X@X.T)@X

def grad_numerico(X):
    eps = 1e-7
    grad = np.zeros(X.shape)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            h = np.zeros(X.shape)
            h[i,j] = eps
            grad[i,j] = (f(X+h) - f(X))/eps
    return grad

# Matriz X inicial
X_initial = rnd.rand(*X_svd.shape)

# Calcular el gradiente analítico
grad_analytical = grad_f(X_initial)

# Calcular el gradiente numérico
grad_numerical = grad_numerico(X_initial)

# Comparar los resultados
print(la.norm(grad_analytical - grad_numerical))

0.00038837619930289066


In [30]:
print(la.norm(grad_analytical))
print(la.norm(grad_numerical))

2539.6835297098855
2539.6837159997767


In [31]:
T = grad_analytical # experimental
E = grad_numerical # teorico
porcentaje_de_error = (la.norm(E - T) / la.norm(T) )*100
porcentaje_de_error

1.5292306886254282e-05

In [32]:
def nesterov_gradient_descent(grad, x_init, alpha, tol=1e-5):
    x, y = x_init.copy(), x_init.copy()
    it = 0

    # Iterar hasta que la norma del gradiente sea menor a la tolerancia establecida
    while np.linalg.norm(grad(y)) > tol:
        # Actualizar iterates
        x_next = y - alpha * grad(y)
        y_next = x_next + (it / (it + 3)) * (x_next - x)

        # Guardar valores
        x, y = x_next, y_next
        # Incrementar contador de iteraciones
        it += 1

    return x, it

X_initial = np.random.rand(n,d)
x, it = nesterov_gradient_descent(grad_f, X_initial, 1e-3)
print('Número de iteraciones: ', it)
print('f(X) = ', f(x))

Número de iteraciones:  293
f(X) =  8045.311152055578


In [35]:
la.norm(X_svd - X_initial)

19.046714670949797

In [22]:
la.norm(X_svd - x)

18.34453082451886

In [23]:
la.norm(f(X_svd) - f(x))

5.4569682106375694e-12